One problem with the automatic fiducial detection is that it is very slow for large fields of view (it can take hours). The slow down comes from the linking across frames. This is problematic because one may need to try different parameters to achieve good fiducial detection.

A possible solution is to visually select regions containing fiducials from histogram plots. Then, the linking can be made to occur only over the search region.

In [1]:
%pylab
import DataSTORM.processors as ds
import pandas as pd
from pathlib import Path

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


### Clean up the data

In [2]:
filename = Path('../test-data/MicroTubules_LargeFOV/FOV1_1500_10ms_1_MMStack_locResults.dat')
with open(str(filename.resolve()), 'r') as file:
    df = pd.read_csv(file, engine = 'c')

/home/douglass/anaconda3/envs/DataSTORM/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
stringMask             = df['uncertainty [nm]'].map(lambda x: isinstance(x, str)).as_matrix()
df['uncertainty [nm]'] = pd.to_numeric(df['uncertainty [nm]'], errors='coerce')
df.replace([np.inf, -np.inf], np.nan, inplace = True)
df.dropna()
df.reindex()
print('Done.')

Done.


# Display 2D histograms
2D histograms seem to be a good way to identify fiducials, especially in microtubules and the centriole samples. This is because the fiducials don't drift much and should be present in every frame, leading to large values in the bins that hide the other features.

Let's first try to generate a nice looking 2D histogram that tells us the information we need. Specifically, we'll try a hexbin because it looks nicer and is slightly more informative than the histogram.

In [6]:
fig, ax = plt.subplots()
ax.hexbin(df['x [nm]'], df['y [nm]'])
plt.show()

In [5]:
plt.close()